<a href="https://colab.research.google.com/github/joshuaalpuerto/ML-guide/blob/main/resume_judger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://medium.com/neuml/judge-your-resume-with-ai-4223a2803509

In [ ]:
!pip install -q pypdf --progress-bar off
!pip install -qU langchain --progress-bar off
!pip install -qU transformers --progress-bar off
# Install AWQ version of model
!pip install -qU vllm --progress-bar off

  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.


In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Store the model in our drive so we can load this model next time faster
# Make sure your drive has free space otherwise comment this.
os.environ['HF_HOME'] = '/content/drive/MyDrive/huggingface'
os.environ['TRANSFORMERS_CACHE'] = '/content/drive/MyDrive/huggingface/models'

In [ ]:
from langchain.llms import VLLM

llm = VLLM(
    model="TheBloke/Starling-LM-7B-alpha-AWQ",
    trust_remote_code=True,  # mandatory for hf models
    max_new_tokens=4096,
    top_k=10,
    top_p=0.95,
    temperature=0.8,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


WARNING 12-16 15:28:35 config.py:173] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 12-16 15:28:35 llm_engine.py:73] Initializing an LLM engine with config: model='TheBloke/Starling-LM-7B-alpha-AWQ', tokenizer='TheBloke/Starling-LM-7B-alpha-AWQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=awq, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 12-16 15:29:47 llm_engine.py:222] # GPU blocks: 1606, # CPU blocks: 2048


In [ ]:
import langchain
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
set_debug(False)

In [ ]:
from google.colab import files
resume = files.upload()

Saving Alpuerto_Joshua_C_CV.pdf to Alpuerto_Joshua_C_CV.pdf


In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Alpuerto_Joshua_C_CV.pdf")
pages = loader.load_and_split()

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnableLambda


# function that will allow us to run this to a langchain expression
def llm_lambda(prompt):
    return llm.generate(prompts=[prompt.text], max_tokens=4000)


def vllm_output_parser(llm_results):
    # vllm generate list of generated result but on our usecase we only need one
    return llm_results.generations[0][0].text


llm_component = RunnableLambda(llm_lambda)
vllm_output_parser_component = RunnableLambda(vllm_output_parser)

In [ ]:
prompt = PromptTemplate.from_template(
    """GPT4 User:
You are a friendly assistant. You answer questions from users.

Answer the following question using only the context below.
Only include information specifically discussed.

question: {question}
context: {context} <|end_of_turn|>GPT4 Assistant:
"""
)

questions = [
    "Who is Joshua?",
    "When did Joshua started working for Jobbatical?",
    "When did Joshua started working for Offshorly and when did it end?",

]

context = [page.page_content for page in pages]
runnable = prompt | llm_component | vllm_output_parser_component

for question in questions:
  runnable.invoke({"question": question, "context": context})

In [ ]:
prompt = PromptTemplate.from_template(
    """GPT4 User:
Base on the context below extract the following information in JSON format. If you can't extract information just put `null`.
>>> JSON output
{{
  firstName: "First name of the applicant",
  lastName: "Last name of the applicant",
  birthday: "Birthday of the applicant",
  currentCompany: "Given the work experiences of the applicant, extract what is the recent or current company the applicant is employed",
  currentPosition: "Given the work experiences of the applicant, extract what is the recent or current job position of the applicant",
  nationality: "Nationality of the applicant",
  yearsOfExperience: "Given the work experiences of the applicant, get the sum of the duration of each `start date` and `end date` of work experience"

}}

context: {context} <|end_of_turn|>GPT4 Assistant:
"""
)

context = [page.page_content for page in pages]
runnable = prompt | llm_component | vllm_output_parser_component

runnable.invoke({"context": context})

Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.89s/it]


'{\n  "firstName": "Joshua",\n  "lastName": "Alpuerto",\n  "birthday": "September 16, 1989",\n  "currentCompany": "Jobbatical, Tallinn, Estonia",\n  "currentPosition": "Staff Engineer",\n  "nationality": "Filipino",\n  "yearsOfExperience": "10"\n}'